# 1) Introduction
The problem that I will be investigating is determining the most suitable location for building a new gastro restaurant. A finacial investor as hired me, being a free-lancer, to investigate this problem. Not only will I be looking for the right area, but I will also be investigating what kind of restaurants are out there. The city of interest for the investor is Copenhagen, Denmark. Copenhagen has a population roughly around 700,000, and is the captial city of Denmark. With various industrial companies having offices around the central area, several tourist attractions and growing modern constructing buildings which directs other danish people from other cities in wanting to live here, copenhagen is an attractive area for opening up a restaurant. The investor has family connections to Denmark, and also has a house in the suburbs. It has always been an interest for him to open up a restaurant because of his passion for food. However, he wants to venture off in new explored areas in the food business, and does not want to open up a place that is already out there. This is why it is very crucial to investigate what type, and how many resturants are in the copenhagen area. Lastly, money is not an issue for him, so finding a location in an area where the rent is very high will not be a problem.

# 2) Data
To solve this problem, I will use foursquare to retrieve data on how many resturants are established in the copenhagen area. This will help me start to determine the possible candidates for location. At the same time, I will be using the data to see what type and how many categories are in the copenhagen city. The data will be scraped, and then a table will be constructed using pandas and beautifulsoup libraries. 